In [1]:
import numpy as np
import pandas as pd
import os
import sys


In [2]:
sys.path.insert(0, '../helpers')
from utils import load_pickle

In [3]:
def bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=5000, upper_percentile=97.5, lower_percentile=2.5, window_size=10.0):
    '''
    To generate 1 bootstrap sample, first sample the 71 subjects to include for this bootstrap sample.
    Then for each included subject, sample the chunk to calculate accuracy for this subject
    '''
    
    rng = np.random.RandomState(0)
    
    num_candidate_subjects = len(candidate_subjects)
    
    bootstrap_accuracy_list = [] # each element is the accuracy of this bootstrap sample (the average accuracy of the selected subjects with their selected chunks in this bootstrap sample)
    for i in range(num_bootstrap_samples):
        print('sample: {}'.format(i))
        #sample the subjects to include for this sample
        subject_location_ix = np.array(range(num_candidate_subjects))
        bootstrap_subject_location_ix = rng.choice(subject_location_ix, num_candidate_subjects, replace=True)
        bootstrap_subjects = candidate_subjects[bootstrap_subject_location_ix]
#         print('subject to include for this sample: {}'.format(bootstrap_subjects))
        
        #for each selected subject, independently resample the chunks to include (as the test set for this subject)
        subject_accuracies = []
        for subject_id in bootstrap_subjects:
            #load the test predictions (for the selected hyper setting) of this subject, and the corresponding true labels
            EnsembleResultSaveDict_this_subject_path = lookup_table.loc[lookup_table['subject_id']==subject_id].experiment_folder.values[0]
            EnsembleResultSaveDict_this_subject = load_pickle(EnsembleResultSaveDict_this_subject_path, 'predictions/ensemble_result_save_dict.pkl')

            EnsembleTestLogits_this_subject = EnsembleResultSaveDict_this_subject['ensemble_test_logits']
            TrueLabels_this_subject = EnsembleResultSaveDict_this_subject['true_labels']

            #bootstrap the chunks to include for this subject (at this bootstrap sample)
            chunk_location_ix = np.array(range(len(TrueLabels_this_subject)))
            bootstrap_chunk_location_ix = rng.choice(chunk_location_ix, len(TrueLabels_this_subject), replace=True)
            bootstrap_chunks_logits = EnsembleTestLogits_this_subject[bootstrap_chunk_location_ix]
            bootstrap_chunks_true_labels = TrueLabels_this_subject[bootstrap_chunk_location_ix]

            accuracy_this_subject = (bootstrap_chunks_logits.argmax(1) == bootstrap_chunks_true_labels).mean()*100

            subject_accuracies.append(accuracy_this_subject)
        
        
        average_accuracy_this_bootstrap_sample = np.mean(np.array(subject_accuracies))
        bootstrap_accuracy_list.append(average_accuracy_this_bootstrap_sample)
    
    bootstrap_accuracy_array = np.array(bootstrap_accuracy_list)
    
    #create bootstrap_accuracy_df for input of the bootstrap plotting code by Mike
    bootstrap_accuracy_df = pd.DataFrame()
    bootstrap_accuracy_df['sample_id'] = np.arange(num_bootstrap_samples)
    bootstrap_accuracy_df['accuracy'] = bootstrap_accuracy_array
    bootstrap_accuracy_df['window_size'] = window_size
    
    accuracy_upper_percentile = np.percentile(bootstrap_accuracy_array, upper_percentile)
    accuracy_lower_percentile = np.percentile(bootstrap_accuracy_array, lower_percentile)

    return accuracy_upper_percentile, accuracy_lower_percentile, bootstrap_accuracy_df




## Logistic Regression

### For each window size, identify the selected hyperparameter setting for each subject using the AllSubjects_summary.csv


In [4]:
root_path = '/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDataSet2021/experiments/SelectWindowSize/LogisticRegression/binary/'


In [5]:
size2sec_10ts_summary_path = 'size_2sec_10ts_stride_3ts_summary/'
size5sec_25ts_summary_path = 'size_5sec_25ts_stride_3ts_summary/'
size10sec_50ts_summary_path = 'size_10sec_50ts_stride_3ts_summary/'
size20sec_100ts_summary_path = 'size_20sec_100ts_stride_3ts_summary/'
size30sec_150ts_summary_path = 'size_30sec_150ts_stride_3ts_summary/'
size40sec_200ts_summary_path = 'size_40sec_200ts_stride_3ts_summary/'


In [6]:
size2sec_df = pd.read_csv(os.path.join(root_path, size2sec_10ts_summary_path, 'AllSubjects_summary.csv'))
size5sec_df = pd.read_csv(os.path.join(root_path, size5sec_25ts_summary_path, 'AllSubjects_summary.csv'))
size10sec_df = pd.read_csv(os.path.join(root_path, size10sec_50ts_summary_path, 'AllSubjects_summary.csv'))
size20sec_df = pd.read_csv(os.path.join(root_path, size20sec_100ts_summary_path, 'AllSubjects_summary.csv'))
size30sec_df = pd.read_csv(os.path.join(root_path, size30sec_150ts_summary_path, 'AllSubjects_summary.csv'))
size40sec_df = pd.read_csv(os.path.join(root_path, size40sec_200ts_summary_path, 'AllSubjects_summary.csv'))

In [7]:
size2sec_df

subject_id  max_validation_accuracy  corresponding_test_accuracy  \
0            1                   96.937                    71.381579   
1           13                   73.258                    58.881579   
2           14                   62.669                    57.894737   
3           15                   77.484                    50.328947   
4           20                   76.620                    61.842105   
..         ...                      ...                          ...   
66          92                   81.311                    53.289474   
67          93                   80.404                    38.486842   
68          94                   88.480                    51.973684   
69          95                   68.348                    59.868421   
70          97                   81.507                    13.815789   

                                   performance_string  \
0   Average cv validation accuracy: 96.93693693693...   
1   Average cv validation accuracy: 73.25772200772...   
2   Average cv validation accuracy: 62.66891891891...   
3   Average cv validation accuracy: 77.48391248391...   
4   Average cv validation accuracy: 76.62001287001...   
..                                                ...   
66  Average cv validation accuracy: 81.31113256113...   
67  Average cv validation accuracy: 80.40379665379...   
68  Average cv validation accuracy: 88.47972972972...   
69  Average cv validation accuracy: 68.34781209781...   
70  Average cv validation accuracy: 81.50740025740...   

                                    experiment_folder  
0   /cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...  
1   /cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...  
2   /cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...  
3   /cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...  
4   /cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...  
..                                                ...  
66  /cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...  
67  /cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...  
68  /cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...  
69  /cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...  
70  /cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...  

[71 rows x 5 columns]

### Get 97.5 and 2.5 percentile using bootstrapping

To generate 1 bootstrap sample, first sample the 71 subjects to include for this bootstrap sample.
Then for each included subject, sample the chunk to calculate accuracy for this subject

In [8]:
num_bootstrap_samples=5000
upper_percentile=97.5
lower_percentile=2.5

candidate_subjects = np.array([1, 13, 14, 15, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 
                      31, 32, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46,
                      47, 48, 49, 5, 51, 52, 54, 55, 56, 57, 58, 60, 61, 62, 63,
                      64, 65, 67, 68, 69, 7, 70, 71, 72, 73, 74, 75, 76, 78, 79,
                      80, 81, 82, 83, 84, 85, 86, 91, 92, 93, 94, 95, 97])


In [9]:
len(candidate_subjects)

71

#### Size2sec_10ts

In [10]:
window_size=10.0
lookup_table = size2sec_df.copy()
size2sec_upper_percentile, size2sec_lower_percentile, size2sec_bootstrap_accuracy_df = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile, window_size=window_size)
print('size2sec_10ts:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, size2sec_upper_percentile, lower_percentile, size2sec_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
sample: 776
sample: 777
samp

sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423
sample: 1424
sample: 1425
sample: 1426

sample: 1985
sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058
sample: 2059
sample: 2060
sample: 2061

sample: 2620
sample: 2621
sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693
sample: 2694
sample: 2695
sample: 2696

sample: 3255
sample: 3256
sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328
sample: 3329
sample: 3330
sample: 3331

sample: 3890
sample: 3891
sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963
sample: 3964
sample: 3965
sample: 3966

sample: 4525
sample: 4526
sample: 4527
sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597
sample: 4598
sample: 4599
sample: 4600
sample: 4601

In [11]:
size2sec_bootstrap_accuracy_df

sample_id   accuracy  window_size
0             0  54.684025         10.0
1             1  55.772795         10.0
2             2  58.784285         10.0
3             3  55.791327         10.0
4             4  53.358970         10.0
...         ...        ...          ...
4995       4995  56.931060         10.0
4996       4996  56.653076         10.0
4997       4997  56.578947         10.0
4998       4998  54.693291         10.0
4999       4999  57.426798         10.0

[5000 rows x 3 columns]

In [12]:
size2sec_bootstrap_accuracy_df.to_csv('./bootstrap_accuracy_csv/LR/size2sec_bootstrap_accuracy.csv', index=False)

#### Size5sec_25ts

In [10]:
window_size=25.0
lookup_table = size5sec_df.copy()
size5sec_upper_percentile, size5sec_lower_percentile, size5sec_bootstrap_accuracy_df = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile, window_size=window_size)
print('size5sec_25ts:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, size5sec_upper_percentile, lower_percentile, size5sec_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 694
sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
sample: 776
samp

sample: 1349
sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423
sample: 1424
sample: 1425

sample: 1984
sample: 1985
sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058
sample: 2059
sample: 2060

sample: 2619
sample: 2620
sample: 2621
sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693
sample: 2694
sample: 2695

sample: 3254
sample: 3255
sample: 3256
sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328
sample: 3329
sample: 3330

sample: 3889
sample: 3890
sample: 3891
sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963
sample: 3964
sample: 3965

sample: 4524
sample: 4525
sample: 4526
sample: 4527
sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597
sample: 4598
sample: 4599
sample: 4600

In [11]:
size5sec_bootstrap_accuracy_df

sample_id   accuracy  window_size
0             0  55.263158         25.0
1             1  57.134915         25.0
2             2  59.673832         25.0
3             3  57.065419         25.0
4             4  54.475537         25.0
...         ...        ...          ...
4995       4995  58.714789         25.0
4996       4996  58.098592         25.0
4997       4997  59.154930         25.0
4998       4998  57.037620         25.0
4999       4999  58.626761         25.0

[5000 rows x 3 columns]

In [13]:
size5sec_bootstrap_accuracy_df.to_csv('./bootstrap_accuracy_csv/LR/size5sec_bootstrap_accuracy.csv', index=False)

#### Size10sec_50ts

In [14]:
window_size=50.0
lookup_table = size10sec_df.copy()
size10sec_upper_percentile, size10sec_lower_percentile, size10sec_bootstrap_accuracy_df = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile, window_size=window_size)
print('size10sec_50ts:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, size10sec_upper_percentile, lower_percentile, size10sec_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
sample: 776
sample: 777
samp

sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423
sample: 1424
sample: 1425
sample: 1426

sample: 1985
sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058
sample: 2059
sample: 2060
sample: 2061

sample: 2620
sample: 2621
sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693
sample: 2694
sample: 2695
sample: 2696

sample: 3255
sample: 3256
sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328
sample: 3329
sample: 3330
sample: 3331

sample: 3890
sample: 3891
sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963
sample: 3964
sample: 3965
sample: 3966

sample: 4525
sample: 4526
sample: 4527
sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597
sample: 4598
sample: 4599
sample: 4600
sample: 4601

In [15]:
size10sec_bootstrap_accuracy_df

sample_id   accuracy  window_size
0             0  55.063936         50.0
1             1  56.768903         50.0
2             2  60.707005         50.0
3             3  57.931801         50.0
4             4  55.656968         50.0
...         ...        ...          ...
4995       4995  60.391957         50.0
4996       4996  57.463862         50.0
4997       4997  58.163454         50.0
4998       4998  58.103225         50.0
4999       4999  58.376575         50.0

[5000 rows x 3 columns]

In [16]:
size10sec_bootstrap_accuracy_df.to_csv('./bootstrap_accuracy_csv/LR/size10sec_bootstrap_accuracy.csv', index=False)


#### Size20sec_100ts

In [17]:
window_size=100.0
lookup_table = size20sec_df.copy()
size20sec_upper_percentile, size20sec_lower_percentile, size20sec_bootstrap_accuracy_df = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile, window_size=window_size)
print('size20sec_100ts:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, size20sec_upper_percentile, lower_percentile, size20sec_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 693
sample: 694
sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
samp

sample: 1348
sample: 1349
sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423
sample: 1424

sample: 1983
sample: 1984
sample: 1985
sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058
sample: 2059

sample: 2618
sample: 2619
sample: 2620
sample: 2621
sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693
sample: 2694

sample: 3249
sample: 3250
sample: 3251
sample: 3252
sample: 3253
sample: 3254
sample: 3255
sample: 3256
sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325

sample: 3884
sample: 3885
sample: 3886
sample: 3887
sample: 3888
sample: 3889
sample: 3890
sample: 3891
sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960

sample: 4519
sample: 4520
sample: 4521
sample: 4522
sample: 4523
sample: 4524
sample: 4525
sample: 4526
sample: 4527
sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595

In [18]:
size20sec_bootstrap_accuracy_df

sample_id   accuracy  window_size
0             0  59.261490        100.0
1             1  58.307079        100.0
2             2  61.086916        100.0
3             3  58.242216        100.0
4             4  57.588955        100.0
...         ...        ...          ...
4995       4995  58.654559        100.0
4996       4996  57.065419        100.0
4997       4997  59.581171        100.0
4998       4998  60.225167        100.0
4999       4999  60.693106        100.0

[5000 rows x 3 columns]

In [20]:
size20sec_bootstrap_accuracy_df.to_csv('./bootstrap_accuracy_csv/LR/size20sec_bootstrap_accuracy.csv', index=False)


#### Size30sec_150ts

In [21]:
window_size=150.0
lookup_table = size30sec_df.copy()
size30sec_upper_percentile, size30sec_lower_percentile, size30sec_bootstrap_accuracy_df = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile, window_size=window_size)
print('size30sec_150ts:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, size30sec_upper_percentile, lower_percentile, size30sec_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 694
sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
sample: 776
samp

sample: 1348
sample: 1349
sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423
sample: 1424

sample: 1983
sample: 1984
sample: 1985
sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058
sample: 2059

sample: 2618
sample: 2619
sample: 2620
sample: 2621
sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693
sample: 2694

sample: 3253
sample: 3254
sample: 3255
sample: 3256
sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328
sample: 3329

sample: 3888
sample: 3889
sample: 3890
sample: 3891
sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963
sample: 3964

sample: 4522
sample: 4523
sample: 4524
sample: 4525
sample: 4526
sample: 4527
sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597
sample: 4598

In [22]:
size30sec_bootstrap_accuracy_df

sample_id   accuracy  window_size
0             0  59.585804        150.0
1             1  58.612861        150.0
2             2  63.255189        150.0
3             3  59.039103        150.0
4             4  58.714789        150.0
...         ...        ...          ...
4995       4995  60.813566        150.0
4996       4996  58.154188        150.0
4997       4997  60.215901        150.0
4998       4998  60.651408        150.0
4999       4999  61.244440        150.0

[5000 rows x 3 columns]

In [23]:
size30sec_bootstrap_accuracy_df.to_csv('./bootstrap_accuracy_csv/LR/size30sec_bootstrap_accuracy.csv', index=False)


#### Size40sec_200ts

In [24]:
window_size=200.0
lookup_table = size40sec_df.copy()
size40sec_upper_percentile, size40sec_lower_percentile, size40sec_bootstrap_accuracy_df = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile, window_size=window_size)
print('size40sec_200ts:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, size40sec_upper_percentile, lower_percentile, size40sec_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 694
sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
sample: 776
samp

sample: 1349
sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423
sample: 1424
sample: 1425

sample: 1984
sample: 1985
sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058
sample: 2059
sample: 2060

sample: 2619
sample: 2620
sample: 2621
sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693
sample: 2694
sample: 2695

sample: 3254
sample: 3255
sample: 3256
sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328
sample: 3329
sample: 3330

sample: 3889
sample: 3890
sample: 3891
sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963
sample: 3964
sample: 3965

sample: 4524
sample: 4525
sample: 4526
sample: 4527
sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597
sample: 4598
sample: 4599
sample: 4600

In [25]:
size40sec_bootstrap_accuracy_df

sample_id   accuracy  window_size
0             0  59.998147        200.0
1             1  57.389733        200.0
2             2  59.780393        200.0
3             3  56.333395        200.0
4             4  58.524833        200.0
...         ...        ...          ...
4995       4995  60.303929        200.0
4996       4996  57.746479        200.0
4997       4997  59.479244        200.0
4998       4998  59.951816        200.0
4999       4999  58.909377        200.0

[5000 rows x 3 columns]

In [26]:
size40sec_bootstrap_accuracy_df.to_csv('./bootstrap_accuracy_csv/LR/size40sec_bootstrap_accuracy.csv', index=False)


## Random Forest

### For each window size, identify the selected hyperparameter setting for each subject using the AllSubjects_summary.csv


In [4]:
root_path = '/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDataSet2021/experiments/SelectWindowSize/RandomForest/binary/'


In [5]:
size2sec_10ts_summary_path = 'size_2sec_10ts_stride_3ts_summary/'
size5sec_25ts_summary_path = 'size_5sec_25ts_stride_3ts_summary/'
size10sec_50ts_summary_path = 'size_10sec_50ts_stride_3ts_summary/'
size20sec_100ts_summary_path = 'size_20sec_100ts_stride_3ts_summary/'
size30sec_150ts_summary_path = 'size_30sec_150ts_stride_3ts_summary/'
size40sec_200ts_summary_path = 'size_40sec_200ts_stride_3ts_summary/'


In [6]:
size2sec_df = pd.read_csv(os.path.join(root_path, size2sec_10ts_summary_path, 'AllSubjects_summary.csv'))
size5sec_df = pd.read_csv(os.path.join(root_path, size5sec_25ts_summary_path, 'AllSubjects_summary.csv'))
size10sec_df = pd.read_csv(os.path.join(root_path, size10sec_50ts_summary_path, 'AllSubjects_summary.csv'))
size20sec_df = pd.read_csv(os.path.join(root_path, size20sec_100ts_summary_path, 'AllSubjects_summary.csv'))
size30sec_df = pd.read_csv(os.path.join(root_path, size30sec_150ts_summary_path, 'AllSubjects_summary.csv'))
size40sec_df = pd.read_csv(os.path.join(root_path, size40sec_200ts_summary_path, 'AllSubjects_summary.csv'))

In [7]:
size2sec_df

subject_id  max_validation_accuracy  corresponding_test_accuracy  \
0            1                   94.234                    72.368421   
1           13                   78.489                    59.539474   
2           14                   51.308                    50.986842   
3           15                   71.712                    44.078947   
4           20                   81.819                    71.052632   
..         ...                      ...                          ...   
66          92                   87.770                    50.000000   
67          93                   69.102                    26.973684   
68          94                   94.784                    45.065789   
69          95                   57.531                    62.171053   
70          97                   81.174                    39.144737   

                                   performance_string  \
0   Average cv validation accuracy: 94.23423423423...   
1   Average cv validation accuracy: 78.48938223938...   
2   Average cv validation accuracy: 51.30791505791...   
3   Average cv validation accuracy: 71.71171171171...   
4   Average cv validation accuracy: 81.81949806949...   
..                                                ...   
66  Average cv validation accuracy: 87.77027027027...   
67  Average cv validation accuracy: 69.10231660231...   
68  Average cv validation accuracy: 94.78442728442...   
69  Average cv validation accuracy: 57.53056628056...   
70  Average cv validation accuracy: 81.17438867438...   

                                    experiment_folder  
0   /cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...  
1   /cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...  
2   /cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...  
3   /cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...  
4   /cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...  
..                                                ...  
66  /cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...  
67  /cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...  
68  /cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...  
69  /cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...  
70  /cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...  

[71 rows x 5 columns]

### Get 97.5 and 2.5 percentile using bootstrapping

To generate 1 bootstrap sample, first sample the 71 subjects to include for this bootstrap sample.
Then for each included subject, sample the chunk to calculate accuracy for this subject

In [8]:
num_bootstrap_samples=5000
upper_percentile=97.5
lower_percentile=2.5

candidate_subjects = np.array([1, 13, 14, 15, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 
                      31, 32, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46,
                      47, 48, 49, 5, 51, 52, 54, 55, 56, 57, 58, 60, 61, 62, 63,
                      64, 65, 67, 68, 69, 7, 70, 71, 72, 73, 74, 75, 76, 78, 79,
                      80, 81, 82, 83, 84, 85, 86, 91, 92, 93, 94, 95, 97])


In [9]:
len(candidate_subjects)

71

#### Size2sec_10ts

In [10]:
window_size=10.0
lookup_table = size2sec_df.copy()
size2sec_upper_percentile, size2sec_lower_percentile, size2sec_bootstrap_accuracy_df = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile, window_size=window_size)
print('size2sec_10ts:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, size2sec_upper_percentile, lower_percentile, size2sec_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 693
sample: 694
sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
samp

sample: 1348
sample: 1349
sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423
sample: 1424

sample: 1982
sample: 1983
sample: 1984
sample: 1985
sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058

sample: 2617
sample: 2618
sample: 2619
sample: 2620
sample: 2621
sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693

sample: 3252
sample: 3253
sample: 3254
sample: 3255
sample: 3256
sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328

sample: 3887
sample: 3888
sample: 3889
sample: 3890
sample: 3891
sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963

sample: 4522
sample: 4523
sample: 4524
sample: 4525
sample: 4526
sample: 4527
sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597
sample: 4598

In [11]:
size2sec_bootstrap_accuracy_df

sample_id   accuracy  window_size
0             0  54.447739         10.0
1             1  55.434581         10.0
2             2  57.783543         10.0
3             3  55.480912         10.0
4             4  53.873239         10.0
...         ...        ...          ...
4995       4995  53.780578         10.0
4996       4996  55.749629         10.0
4997       4997  55.842291         10.0
4998       4998  55.837658         10.0
4999       4999  56.759637         10.0

[5000 rows x 3 columns]

In [12]:
size2sec_bootstrap_accuracy_df.to_csv('./bootstrap_accuracy_csv/RF/size2sec_bootstrap_accuracy.csv', index=False)


#### Size5sec_25ts

In [13]:
window_size=25.0
lookup_table = size5sec_df.copy()
size5sec_upper_percentile, size5sec_lower_percentile, size5sec_bootstrap_accuracy_df = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile, window_size=window_size)
print('size5sec_25ts:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, size5sec_upper_percentile, lower_percentile, size5sec_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 693
sample: 694
sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
samp

sample: 1348
sample: 1349
sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423
sample: 1424

sample: 1983
sample: 1984
sample: 1985
sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058
sample: 2059

sample: 2618
sample: 2619
sample: 2620
sample: 2621
sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693
sample: 2694

sample: 3253
sample: 3254
sample: 3255
sample: 3256
sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328
sample: 3329

sample: 3888
sample: 3889
sample: 3890
sample: 3891
sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963
sample: 3964

sample: 4523
sample: 4524
sample: 4525
sample: 4526
sample: 4527
sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597
sample: 4598
sample: 4599

In [14]:
size5sec_bootstrap_accuracy_df

sample_id   accuracy  window_size
0             0  54.322646         25.0
1             1  55.077835         25.0
2             2  58.571164         25.0
3             3  55.610638         25.0
4             4  52.728873         25.0
...         ...        ...          ...
4995       4995  54.081727         25.0
4996       4996  54.730356         25.0
4997       4997  54.480170         25.0
4998       4998  54.058562         25.0
4999       4999  55.934952         25.0

[5000 rows x 3 columns]

In [15]:
size5sec_bootstrap_accuracy_df.to_csv('./bootstrap_accuracy_csv/RF/size5sec_bootstrap_accuracy.csv', index=False)

#### Size10sec_50ts

In [16]:
window_size=50.0
lookup_table = size10sec_df.copy()
size10sec_upper_percentile, size10sec_lower_percentile, size10sec_bootstrap_accuracy_df = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile, window_size=window_size)
print('size10sec_50ts:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, size10sec_upper_percentile, lower_percentile, size10sec_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 692
sample: 693
sample: 694
sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
samp

sample: 1347
sample: 1348
sample: 1349
sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423

sample: 1982
sample: 1983
sample: 1984
sample: 1985
sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058

sample: 2617
sample: 2618
sample: 2619
sample: 2620
sample: 2621
sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693

sample: 3251
sample: 3252
sample: 3253
sample: 3254
sample: 3255
sample: 3256
sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327

sample: 3886
sample: 3887
sample: 3888
sample: 3889
sample: 3890
sample: 3891
sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962

sample: 4521
sample: 4522
sample: 4523
sample: 4524
sample: 4525
sample: 4526
sample: 4527
sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597

In [17]:
size10sec_bootstrap_accuracy_df

sample_id   accuracy  window_size
0             0  53.678651         50.0
1             1  54.772053         50.0
2             2  57.755745         50.0
3             3  56.351927         50.0
4             4  53.030022         50.0
...         ...        ...          ...
4995       4995  53.312639         50.0
4996       4996  53.062454         50.0
4997       4997  55.277057         50.0
4998       4998  54.132691         50.0
4999       4999  55.782061         50.0

[5000 rows x 3 columns]

In [18]:
size10sec_bootstrap_accuracy_df.to_csv('./bootstrap_accuracy_csv/RF/size10sec_bootstrap_accuracy.csv', index=False)


#### Size20sec_100ts

In [19]:
window_size=100.0
lookup_table = size20sec_df.copy()
size20sec_upper_percentile, size20sec_lower_percentile, size20sec_bootstrap_accuracy_df = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile, window_size=window_size)
print('size20sec_100ts:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, size20sec_upper_percentile, lower_percentile, size20sec_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 693
sample: 694
sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
samp

sample: 1348
sample: 1349
sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423
sample: 1424

sample: 1982
sample: 1983
sample: 1984
sample: 1985
sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058

sample: 2617
sample: 2618
sample: 2619
sample: 2620
sample: 2621
sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693

sample: 3252
sample: 3253
sample: 3254
sample: 3255
sample: 3256
sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328

sample: 3887
sample: 3888
sample: 3889
sample: 3890
sample: 3891
sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963

sample: 4521
sample: 4522
sample: 4523
sample: 4524
sample: 4525
sample: 4526
sample: 4527
sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597

In [20]:
size20sec_bootstrap_accuracy_df

sample_id   accuracy  window_size
0             0  54.776686        100.0
1             1  55.397517        100.0
2             2  57.542624        100.0
3             3  55.555041        100.0
4             4  53.623054        100.0
...         ...        ...          ...
4995       4995  55.953484        100.0
4996       4996  52.668643        100.0
4997       4997  53.349703        100.0
4998       4998  54.160489        100.0
4999       4999  55.365085        100.0

[5000 rows x 3 columns]

In [21]:
size20sec_bootstrap_accuracy_df.to_csv('./bootstrap_accuracy_csv/RF/size20sec_bootstrap_accuracy.csv', index=False)


#### Size30sec_150ts

In [22]:
window_size=150.0
lookup_table = size30sec_df.copy()
size30sec_upper_percentile, size30sec_lower_percentile, size30sec_bootstrap_accuracy_df = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile, window_size=window_size)
print('size30sec_150ts:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, size30sec_upper_percentile, lower_percentile, size30sec_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 693
sample: 694
sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
samp

sample: 1348
sample: 1349
sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423
sample: 1424

sample: 1982
sample: 1983
sample: 1984
sample: 1985
sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058

sample: 2617
sample: 2618
sample: 2619
sample: 2620
sample: 2621
sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693

sample: 3252
sample: 3253
sample: 3254
sample: 3255
sample: 3256
sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328

sample: 3887
sample: 3888
sample: 3889
sample: 3890
sample: 3891
sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963

sample: 4522
sample: 4523
sample: 4524
sample: 4525
sample: 4526
sample: 4527
sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597
sample: 4598

In [23]:
size30sec_bootstrap_accuracy_df

sample_id   accuracy  window_size
0             0  55.791327        150.0
1             1  56.245367        150.0
2             2  57.996664        150.0
3             3  57.602854        150.0
4             4  54.781319        150.0
...         ...        ...          ...
4995       4995  58.251483        150.0
4996       4996  54.924944        150.0
4997       4997  56.467754        150.0
4998       4998  56.625278        150.0
4999       4999  57.602854        150.0

[5000 rows x 3 columns]

In [24]:
size30sec_bootstrap_accuracy_df.to_csv('./bootstrap_accuracy_csv/RF/size30sec_bootstrap_accuracy.csv', index=False)


#### Size40sec_200ts

In [25]:
window_size = 200.0
lookup_table = size40sec_df.copy()
size40sec_upper_percentile, size40sec_lower_percentile, size40sec_bootstrap_accuracy_df = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile, window_size=window_size)
print('size40sec_200ts:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, size40sec_upper_percentile, lower_percentile, size40sec_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 693
sample: 694
sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
samp

sample: 1347
sample: 1348
sample: 1349
sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423

sample: 1982
sample: 1983
sample: 1984
sample: 1985
sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058

sample: 2617
sample: 2618
sample: 2619
sample: 2620
sample: 2621
sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693

sample: 3252
sample: 3253
sample: 3254
sample: 3255
sample: 3256
sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328

sample: 3887
sample: 3888
sample: 3889
sample: 3890
sample: 3891
sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963

sample: 4521
sample: 4522
sample: 4523
sample: 4524
sample: 4525
sample: 4526
sample: 4527
sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597

In [26]:
size40sec_bootstrap_accuracy_df

sample_id   accuracy  window_size
0             0  57.528725        200.0
1             1  57.273907        200.0
2             2  59.280022        200.0
3             3  57.449963        200.0
4             4  57.028354        200.0
...         ...        ...          ...
4995       4995  59.627502        200.0
4996       4996  56.324129        200.0
4997       4997  56.416790        200.0
4998       4998  57.908636        200.0
4999       4999  59.261490        200.0

[5000 rows x 3 columns]

In [27]:
size40sec_bootstrap_accuracy_df.to_csv('./bootstrap_accuracy_csv/RF/size40sec_bootstrap_accuracy.csv', index=False)
